# Julia for (scientific) python programmers

Generally we're going to assume you've had a look at an introductory Julia article such as [Learn Julia in Y minutes](http://learnxinyminutes.com/docs/julia/ "Learn Julia in Y minutes").

This article was written using the 0.5 dev branch of Julia.


### Multiple dispatch

The biggest difference for me coming to Julia from python is the type system. I have used statically typed languages such as C, but not interactively.

In python methods and data are wrapped together in classes. For example, suppose we want a 2D point class which supports use of ```+``` for addition. In python our code might look like:

```python
class foo:

    def __init__(self, x, y):
        self.x = x 
        self.y = y
        
    def __add__(self, othr):
        newx = self.x + othr.x
        newy = self.y + othr.y
        return foo(newx, newy)
        
a = foo(1, 2)
b = foo(pi, e)
c = a + b
```         

Where as in Julia we define the data type, and operations for that data type separately. 

In [73]:
import Base.+  #explicitly state where the function + is coming from.

type foo
   x::Float64
   y::Float64
end

function (+)(left::foo, rght::foo)
    newx = left.x + rght.x
    newy = left.y + rght.y
    foo(newx, newy)
end
        
a = foo(1, 2)
b = foo(pi, e)
c = a + b

foo(4.141592653589793,4.718281828459045)

Even if you don't often want to overload operators for your own classes this matters. It matters because it affects how we can look for and interact with the data types and methods found in modules.  

For example, we cannot enter ```dir(foo)``` and expect a list of all the attributes and methods of an object.

This is important because Julia is immature, and documentation outside the core language is incomplete. Yet we still need to be able to discover how modules work.

### What is in a module?

Say we want to work with some sparse matrices on a GPU, so we load up the [```CUSPARSE.jl```](https://github.com/JuliaGPU/CUSPARSE.jl "CUSPARSE") module. 

We'll then look at what sorts of data types and functions it makes available to us.

In [48]:
using CUSPARSE

What exactly have we got by doing this?

In [49]:
typeof(CUSPARSE)

Module

In [64]:
methodswith(typeof(CUSPARSE))[1:10]

10-element Array{Method,1}:
 fullname(m::Module) at reflection.jl:9                                         
 isconst(m::Module, s::Symbol) at reflection.jl:67                              
 methodswith(t::Type{T}, m::Module) at interactiveutil.jl:315                   
 methodswith(t::Type{T}, m::Module, showparents::Bool) at interactiveutil.jl:315
 module_name(m::Module) at reflection.jl:4                                      
 module_parent(m::Module) at reflection.jl:5                                    
 names(m::Module) at reflection.jl:32                                           
 names(m::Module, all::Bool) at reflection.jl:31                                
 names(m::Module, all::Bool, imported::Bool) at reflection.jl:30                
 serialize(s::SerializationState{I<:IO}, m::Module) at serialize.jl:262

We can use the three names functions to give us increasing levels of information about the variables in the module.

First we look for all the variables _exported_ from the package.

In [69]:
n1 = names(CUSPARSE)

5-element Array{Symbol,1}:
 :CUSPARSE           
 :CudaSparseMatrixBSR
 :CudaSparseMatrixCSC
 :CudaSparseMatrixCSR
 :CudaSparseMatrixHYB

Next we get the names of all variables _defined_ in the module. 

In [72]:
n2 = names(CUSPARSE, true); 

println("Yields $(length(n2)) variable names, of which the first ten are:")
n2[1:10]


Yields 175 variable names, of which the first ten are:


10-element Array{Symbol,1}:
 :CUSPARSE                   
 :CUSPARSE_ACTION_NUMERIC    
 :CUSPARSE_ACTION_SYMBOLIC   
 :CUSPARSE_DIAG_TYPE_NON_UNIT
 :CUSPARSE_DIAG_TYPE_UNIT    
 :CUSPARSE_DIRECTION_COL     
 :CUSPARSE_DIRECTION_ROW     
 :CUSPARSE_FILL_MODE_LOWER   
 :CUSPARSE_FILL_MODE_UPPER   
 :CUSPARSE_HYB_PARTITION_AUTO

Next we get the names of all variables _used_ in the package.

In [70]:
n3 = names(CUSPARSE, true, true); 

println("Yields $(length(n3)) variable names, of which the first ten are:")
n3[1:10]

Yields 260 variable names, of which the first ten are:


10-element Array{Symbol,1}:
 :+                   
 :(==)                
 symbol("@eval")      
 :AbstractSparseArray 
 :AbstractSparseMatrix
 :AbstractSparseVector
 :Any                 
 :ArgumentError       
 :Base                
 :CUSPARSE            

The ```whos()``` function gives a variables were exported by the package. Exporting a variable is the primary way of a developer indicating where the gateways into a module are for users. Exported variables can be used _without_ ```modname.varname``` notation, we can just write ```varname```.  In case of namespace conflicts use ```import modname``` rather than ```using modname``` and always use the longer notation.

In [75]:
whos(CUSPARSE)

                      CUSPARSE    731 KB     Module : CUSPARSE
           CudaSparseMatrixBSR    252 bytes  DataType : CUSPARSE.CudaSparseMatr…
           CudaSparseMatrixCSC    228 bytes  DataType : CUSPARSE.CudaSparseMatr…
           CudaSparseMatrixCSR    228 bytes  DataType : CUSPARSE.CudaSparseMatr…
           CudaSparseMatrixHYB    204 bytes  DataType : CUSPARSE.CudaSparseMatr…


### How can we interact with a variable?

Suppose we like the sound of the ```CudaSparseMatrixCSR``` object and want to find out more about it.

In [77]:
typeof(CudaSparseMatrixCSR)

DataType

Some of the things we can do with a data type:

In [79]:
methodswith(DataType)[1:10]

10-element Array{Method,1}:
 deserialize(s::SerializationState{I<:IO}, t::DataType) at serialize.jl:667
 dump(io::IO, x::DataType) at show.jl:971                                  
 dump(io::IO, x::DataType, n::Int64) at show.jl:970                        
 dump(io::IO, x::DataType, n::Int64, indent) at show.jl:969                
 eltype(t::DataType) at operators.jl:190                                   
 fieldnames(t::DataType) at reflection.jl:54                               
 fieldoffsets(x::DataType) at reflection.jl:86                             
 isbits(t::DataType) at reflection.jl:77                                   
 names(t::DataType) at deprecated.jl:49                                    
 serialize(s::SerializationState{I<:IO}, t::DataType) at serialize.jl:377

Now we can look up what attributes the CSR matrix has:

In [26]:
fieldnames(CudaSparseMatrixCSR)

6-element Array{Symbol,1}:
 :rowPtr
 :colVal
 :nzVal 
 :dims  
 :nnz   
 :dev   

What functions accept a ```CudaSparseMatrixCSR``` as an argument?

In [28]:
methodswith(CudaSparseMatrixCSR)

4-element Array{Method,1}:
 copy(Mat::CUSPARSE.CudaSparseMatrixCSR{T}) at /home/mcp50/.julia/v0.5/CUSPARSE/src/util.jl:151                                       
 copy!(dst::CUSPARSE.CudaSparseMatrixCSR{T}, src::CUSPARSE.CudaSparseMatrixCSR{T}) at /home/mcp50/.julia/v0.5/CUSPARSE/src/util.jl:119
 similar(Mat::CUSPARSE.CudaSparseMatrixCSR{T}) at /home/mcp50/.julia/v0.5/CUSPARSE/src/util.jl:104                                    
 to_host{T}(Mat::CUSPARSE.CudaSparseMatrixCSR{T}) at /home/mcp50/.julia/v0.5/CUSPARSE/src/util.jl:76

Unlike python, functions in Julia are not built into classes. 

Instead the multiple dispatch system uses one function name with many variant methods catering for different data types.

In [37]:
methods(CUSPARSE.to_host)

# 3 methods for generic function "to_host":
to_host{T}(g::CUDArt.AbstractCudaArray{T,N}) at /home/mcp50/.julia/v0.5/CUDArt/src/arrays.jl:87
to_host{T}(Mat::CUSPARSE.CudaSparseMatrixCSC{T}) at /home/mcp50/.julia/v0.5/CUSPARSE/src/util.jl:74
to_host{T}(Mat::CUSPARSE.CudaSparseMatrixCSR{T}) at /home/mcp50/.julia/v0.5/CUSPARSE/src/util.jl:76

Here we can see that variant methods exist for sending either an abstract version of a sparse matrix, a compressed sparse column matrix, or a compressed sparse row matrix back to the host.

### Using a variable.

Here's a convenience function I made to search for functions by part of name within a module:

In [90]:
function lookup(modname::Module, srchterm::Regex)
    vars = names(modname, true)
    indx = map((nm)->ismatch(srchterm, string(nm)), vars);
    vars[indx]
end

lookup (generic function with 1 method)

Say we want to look up matrix vector operations:

In [92]:
lookup(CUSPARSE, r"mv")

7-element Array{Symbol,1}:
 :bsrmv    
 :bsrmv!   
 :chkmvdims
 :csrmv    
 :csrmv!   
 :hybmv    
 :hybmv!   

In [96]:
methods(CUSPARSE.csrmv)

# 24 methods for generic function "csrmv":
csrmv(transa::Char, alpha::Float32, A::CUSPARSE.CudaSparseMatrixCSR{Float32}, X::CUDArt.CudaArray{Float32,1}, beta::Float32, Y::CUDArt.CudaArray{Float32,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:659
csrmv(transa::Char, alpha::Float32, A::CUSPARSE.CudaSparseMatrixCSR{Float32}, X::CUDArt.CudaArray{Float32,1}, Y::CUDArt.CudaArray{Float32,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:667
csrmv(transa::Char, A::CUSPARSE.CudaSparseMatrixCSR{Float32}, X::CUDArt.CudaArray{Float32,1}, beta::Float32, Y::CUDArt.CudaArray{Float32,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:675
csrmv(transa::Char, A::CUSPARSE.CudaSparseMatrixCSR{Float32}, X::CUDArt.CudaArray{Float32,1}, Y::CUDArt.CudaArray{Float32,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:682
csrmv(transa::Char, alpha::Float32, A::CUSPARSE.CudaSparseMatrixCSR{Float32}, X::CUDArt.CudaArray{Float32,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:689
csrmv(transa::Char, A::CUSPARSE.CudaSparseMatrixCSR{Float32}, X::CUDArt.CudaArray{Float32,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:695
csrmv(transa::Char, alpha::Float64, A::CUSPARSE.CudaSparseMatrixCSR{Float64}, X::CUDArt.CudaArray{Float64,1}, beta::Float64, Y::CUDArt.CudaArray{Float64,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:659
csrmv(transa::Char, alpha::Float64, A::CUSPARSE.CudaSparseMatrixCSR{Float64}, X::CUDArt.CudaArray{Float64,1}, Y::CUDArt.CudaArray{Float64,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:667
csrmv(transa::Char, A::CUSPARSE.CudaSparseMatrixCSR{Float64}, X::CUDArt.CudaArray{Float64,1}, beta::Float64, Y::CUDArt.CudaArray{Float64,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:675
csrmv(transa::Char, A::CUSPARSE.CudaSparseMatrixCSR{Float64}, X::CUDArt.CudaArray{Float64,1}, Y::CUDArt.CudaArray{Float64,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:682
csrmv(transa::Char, alpha::Float64, A::CUSPARSE.CudaSparseMatrixCSR{Float64}, X::CUDArt.CudaArray{Float64,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:689
csrmv(transa::Char, A::CUSPARSE.CudaSparseMatrixCSR{Float64}, X::CUDArt.CudaArray{Float64,1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:695
csrmv(transa::Char, alpha::Complex{Float32}, A::CUSPARSE.CudaSparseMatrixCSR{Complex{Float32}}, X::CUDArt.CudaArray{Complex{Float32},1}, beta::Complex{Float32}, Y::CUDArt.CudaArray{Complex{Float32},1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:659
csrmv(transa::Char, alpha::Complex{Float32}, A::CUSPARSE.CudaSparseMatrixCSR{Complex{Float32}}, X::CUDArt.CudaArray{Complex{Float32},1}, Y::CUDArt.CudaArray{Complex{Float32},1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:667
csrmv(transa::Char, A::CUSPARSE.CudaSparseMatrixCSR{Complex{Float32}}, X::CUDArt.CudaArray{Complex{Float32},1}, beta::Complex{Float32}, Y::CUDArt.CudaArray{Complex{Float32},1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:675
csrmv(transa::Char, A::CUSPARSE.CudaSparseMatrixCSR{Complex{Float32}}, X::CUDArt.CudaArray{Complex{Float32},1}, Y::CUDArt.CudaArray{Complex{Float32},1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:682
csrmv(transa::Char, alpha::Complex{Float32}, A::CUSPARSE.CudaSparseMatrixCSR{Complex{Float32}}, X::CUDArt.CudaArray{Complex{Float32},1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:689
csrmv(transa::Char, A::CUSPARSE.CudaSparseMatrixCSR{Complex{Float32}}, X::CUDArt.CudaArray{Complex{Float32},1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/src/sparse.jl:695
csrmv(transa::Char, alpha::Complex{Float64}, A::CUSPARSE.CudaSparseMatrixCSR{Complex{Float64}}, X::CUDArt.CudaArray{Complex{Float64},1}, beta::Complex{Float64}, Y::CUDArt.CudaArray{Complex{Float64},1}, index::Char) at /home/mcp50/.julia/v0.5/CUSPARSE/s

Let's go ahead and make some sparse matrices.

In [85]:
# dimensions and fill proportion
N = 20
M = 10
p = 0.1

# create matrices A,B on the CPU 
A = sprand(N,M,p)
B = sprand(N,M,p)

# generate scalar parameters
alpha = rand()
beta  = rand()

# convert A,B to CSR format and
# move them to the GPU - one step
d_A = CudaSparseMatrixCSR(A)
d_B = CudaSparseMatrixCSR(B)

CUSPARSE.CudaSparseMatrixCSR{Float64}(CUDArt.CudaArray{Int32,1}(CUDArt.CudaPtr{Int32}(Ptr{Int32} @0x0000000b037a5400),(21,),0),CUDArt.CudaArray{Int32,1}(CUDArt.CudaPtr{Int32}(Ptr{Int32} @0x0000000b037a5600),(16,),0),CUDArt.CudaArray{Float64,1}(CUDArt.CudaPtr{Float64}(Ptr{Float64} @0x0000000b037a5800),(16,),0),(20,10),16,0)

Then add them togther using ```geam```:

In [87]:
# perform alpha * A + beta * B
d_C = CUSPARSE.geam(alpha, d_A, beta, d_B, 'O', 'O', 'O')

# bring the result back to the CPU
C = CUSPARSE.to_host(d_C)

# observe a zero matrix
alpha*A + beta*B - C

20x10 sparse matrix with 0 Float64 entries: